## Домашнее задание¶

## Подбор оптимальных гиперпараметров для LightFM

- Постройте модели с помощью библиотеки LightFM, изменяя следующие параметры
  - функция потерь, регуляризация
  - количество компонент
  - отдельно постройте модели, используя только матрицу взаимодействий и матрицу взаимодействий + признаки (набор признаков может быть различным, например как на вебинаре)
  
- Посчитайте метрики (Precision@5, MAP@5) для разных наборов гиперпараметров и выберете лучший набор


In [1]:
import numpy as np
import os, sys, itertools
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items


module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

C:\Users\xiaomi\AppData\Roaming\Python\Python39\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data = pd.read_csv('../Lesson_2/retail_train.csv')
item_features = pd.read_csv('../Lesson_2/product.csv')
user_features = pd.read_csv('../Lesson_2/hh_demographic.csv')

data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [5]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


#### Filter items

In [7]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, 5000, item_features)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


#### Prepare data set

In [8]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
# data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]
test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [10]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

#### Prepare user and item features

In [11]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [13]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
def grid_search(sparse_user_item, params, N=5):
    test_data = csr_matrix(test_user_item_matrix).tocsr()
    user_features = csr_matrix(user_feat_lightfm.values).tocsr()
    item_features = csr_matrix(item_feat_lightfm.values).tocsr()
    
    scores = []
    for components, loss, l_rate, alpha in itertools.product(*params.values()):
        model = LightFM(no_components=components,
                loss=loss,
                learning_rate=l_rate, 
                item_alpha=alpha,
                user_alpha=alpha, 
                random_state=42)
        
        model = model.fit((sparse_user_item > 0) * 1,
                           sample_weight=coo_matrix(user_item_matrix),
                           user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                           item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                           epochs=15, 
                           num_threads=6,
                           verbose=False) 
        
        precision = precision_at_k(model,
                                   test_data,
                                   user_features=user_features,
                                   item_features=item_features,
                                   k=N).mean()
                
        score = {'no_components': components,
                 'loss':          loss,
                 'learning_rate': l_rate,
                 'alpha':         alpha,
                 'precision':     precision}
        print(score)
        scores.append(score)
    
    return scores

def best_params(scores, all_=False):
    df = pd.DataFrame(scores, index=range(len(scores))).sort_values('precision', ascending=False)
    if all_:
        return df
    df = df.head(1)
    return df.to_dict()

In [ ]:
%%time

parameters = {'no_components': [10, 100],
              'loss':          ['bpr', 'warp'],
              'learning_rate': [0.001, 0.1],
              'alpha':         [0.01, 0.1]}

scores = grid_search(sparse_user_item, parameters, N=5)

{'no_components': 10, 'loss': 'bpr', 'learning_rate': 0.001, 'alpha': 0.01, 'precision': 0.43852624}
{'no_components': 10, 'loss': 'bpr', 'learning_rate': 0.001, 'alpha': 0.1, 'precision': 0.43852624}
{'no_components': 10, 'loss': 'bpr', 'learning_rate': 0.1, 'alpha': 0.01, 'precision': 0.43852624}
{'no_components': 10, 'loss': 'bpr', 'learning_rate': 0.1, 'alpha': 0.1, 'precision': 0.20937127}
{'no_components': 10, 'loss': 'warp', 'learning_rate': 0.001, 'alpha': 0.01, 'precision': 0.12583101}
{'no_components': 10, 'loss': 'warp', 'learning_rate': 0.001, 'alpha': 0.1, 'precision': 0.1465759}
{'no_components': 10, 'loss': 'warp', 'learning_rate': 0.1, 'alpha': 0.01, 'precision': 0.2867441}
{'no_components': 10, 'loss': 'warp', 'learning_rate': 0.1, 'alpha': 0.1, 'precision': 0.25446537}
{'no_components': 100, 'loss': 'bpr', 'learning_rate': 0.001, 'alpha': 0.01, 'precision': 0.38077694}
{'no_components': 100, 'loss': 'bpr', 'learning_rate': 0.001, 'alpha': 0.1, 'precision': 0.43852624}

In [ ]:
best_params(scores, True)

In [ ]:
best_params(scores)